In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2020-06-21 07:39:42--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 209.85.146.139, 209.85.146.113, 209.85.146.102, ...
Connecting to drive.google.com (drive.google.com)|209.85.146.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/blsqu5tt0ede3qvgsbsphqrrs554ktfe/1592725125000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2020-06-21 07:39:42--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/blsqu5tt0ede3qvgsbsphqrrs554ktfe/1592725125000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 64.233.191.132, 2607:f8b0:4001:c0c::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|64.233

In [4]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('/tmp/sentiment.csv')


In [5]:
print(dataset)

      Unnamed: 0                                               text  sentiment
0              0  So there is no way for me to plug it in here i...          0
1              1                         Good case Excellent value.          1
2              2                             Great for the jawbone.          1
3              3  Tied to charger for conversations lasting more...          0
4              4                                  The mic is great.          1
...          ...                                                ...        ...
1987        1987  I think food should have flavor and texture an...          0
1988        1988                           Appetite instantly gone.          0
1989        1989  Overall I was not impressed and would not go b...          0
1990        1990  The whole experience was underwhelming and I t...          0
1991        1991  Then as if I hadn't wasted enough of my life t...          0

[1992 rows x 3 columns]


In [6]:
sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

In [8]:
training_size = int(len(sentences)*0.8)
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]


training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [9]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [10]:
tokenizer = Tokenizer(num_words=vocab_size , oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences , maxlen=max_length , padding=padding_type , truncating=trunc_type)



In [11]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences , maxlen=max_length , 
                               padding = padding_type , truncating = trunc_type)

In [12]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 9606      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [18]:

num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 0s 4ms/step - loss: 0.6923 - accuracy: 0.5173 - val_loss: 0.7054 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6812 - accuracy: 0.5832 - val_loss: 0.6878 - val_accuracy: 0.5313
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6472 - accuracy: 0.7451 - val_loss: 0.6782 - val_accuracy: 0.5263
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.5756 - accuracy: 0.7960 - val_loss: 0.6101 - val_accuracy: 0.7043
Epoch 5/10
50/50 [==============================] - 0s 3ms/step - loss: 0.4722 - accuracy: 0.8763 - val_loss: 0.5765 - val_accuracy: 0.7068
Epoch 6/10
50/50 [==============================] - 0s 3ms/step - loss: 0.3690 - accuracy: 0.9109 - val_loss: 0.5026 - val_accuracy: 0.7719
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.2767 - accuracy: 0.9353 - val_loss: 0.4815 - val_accuracy: 0.7644
Epoch 8/10
50/50 [==

In [19]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(1000, 16)


In [20]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')
  

In [23]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')


['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.9878398]


I hate spaghetti
[0.10443485]


Everything was cold
[0.3651631]


Everything was hot exactly as I wanted
[0.73138726]


Everything was green
[0.5476241]


the host seated us immediately
[0.80787987]


they gave us free chocolate cake
[0.94289654]


not sure about the wilted flowers on the table
[0.06408933]


only works when I stand on tippy toes
[0.9551733]


does not work when I stand on my head
[0.00271011]


